<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/FF%2B%2B_download_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 기존 미디어파이프 설치 대신 아래 실행
!pip install insightface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071490 sha256=d4d3b3d202ca2c5a32e4e8128f61f23602855ddfb85b34dd6aeca70dcf80c3d8
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [ ]:
import cv2
import numpy as np
from google.colab import drive
import os
import json
import urllib.request
import tempfile
import random
from tqdm import tqdm
from insightface.app import FaceAnalysis

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# 랜덤 시드 고정
SEED = 42
random.seed(SEED)

# --- 설정 ---
GDRIVE_BASE_PATH = "/content/drive/MyDrive/HECTO/Dataset/FF_frames"
NUM_FRAMES = 15
SERVER_URL = "http://kaldir.vc.in.tum.de/faceforensics/v3/"
FILELIST_URL = SERVER_URL + "misc/filelist.json"

# 조작 방식 리스트 (4가지 주요 방식)
MANIPULATIONS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']
COMPRESSION = 'c23'

In [ ]:
def get_balanced_dataset(sample_per_category=250, is_test=False):
    print("서버에서 파일 리스트를 가져오는 중...")
    with urllib.request.urlopen(FILELIST_URL) as url:
        file_pairs = json.loads(url.read().decode())

    # 테스트 모드인 경우 카테고리당 2개만 선택
    if is_test:
        sample_per_category = 2

    # 1. Real 영상 리스트 확보 (전체에서 샘플링)
    # 가짜가 총 4종 * 250 = 1000개이므로, 진짜도 1000개 전체 사용
    # 수정된 리스트 생성부 (셀 3 확인)
    real_list = []
    for pair in file_pairs:
        real_list.extend([pair[0] + '.mp4', pair[1] + '.mp4'])
    # 중복 제거 (혹시 모를 겹침 방지)
    real_list = sorted(list(set(real_list)))

    if is_test:
        real_list = random.sample(real_list, 2)
    else:
        # 가짜 총합에 맞춰 1000개 샘플링 (FF++ 원본은 딱 1000개이므로 셔플만)
        random.shuffle(real_list)

    # 2. 가짜 영상 리스트 확보 (카테고리별 250개씩)
    fake_dict = {}
    for method in MANIPULATIONS:
        # 각 조작 방식별로 랜덤하게 pair 선택
        sampled_pairs = random.sample(file_pairs, sample_per_category)
        fake_dict[method] = ['_'.join(p) + '.mp4' for p in sampled_pairs]

    return real_list, fake_dict


In [ ]:
# RetinaFace 초기화 (랜드마크 추출을 위해 landmark2d 추가)
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

def get_aligned_face(image, face_info, target_size=(256, 256)):
    try:
        h_img, w_img = image.shape[:2]
        landmarks = getattr(face_info, 'kps', None)
        bbox = getattr(face_info, 'bbox', None)
        if bbox is None: return None

        x1, y1, x2, y2 = bbox.astype(int)
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # 1. 정렬 시도 (눈 위치가 있는 경우)
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))

            M = cv2.getRotationMatrix2D((float(center_x), float(center_y)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w_img, h_img), flags=cv2.INTER_CUBIC)

        # 2. 정사각 크롭 영역 계산 (애초에 정사각으로 잡기)
        w_box, h_box = x2 - x1, y2 - y1
        # 얼굴의 더 긴 쪽을 기준으로 마진(30%)을 포함한 정사각 변 길이 결정
        side_length = int(max(w_box, h_box) * 1.3)

        half_side = side_length // 2
        nx1, ny1 = center_x - half_side, center_y - half_side
        nx2, ny2 = center_x + half_side, center_y + half_side

        # 이미지 경계를 벗어나는 경우 패딩 처리 (검은색으로 채움)
        # 이렇게 해야 '짜부'되지 않고 정사각 형태를 유지합니다.
        pad_x1 = max(0, -nx1)
        pad_y1 = max(0, -ny1)
        pad_x2 = max(0, nx2 - w_img)
        pad_y2 = max(0, ny2 - h_img)

        nx1, ny1 = max(0, nx1), max(0, ny1)
        nx2, ny2 = min(w_img, nx2), min(h_img, ny2)

        face_crop = image[ny1:ny2, nx1:nx2]

        # 부족한 부분을 검은색으로 채워 정사각 유지
        if pad_x1 > 0 or pad_y1 > 0 or pad_x2 > 0 or pad_y2 > 0:
            face_crop = cv2.copyMakeBorder(face_crop, pad_y1, pad_y2, pad_x1, pad_x2,
                                           cv2.BORDER_CONSTANT, value=[0, 0, 0])

        return cv2.resize(face_crop, target_size)
    except:
        return None

def resize_with_letterbox(image, target_size=(256, 256)):
    if image.size == 0: return np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    h, w = image.shape[:2]
    scale = min(target_size[0]/w, target_size[1]/h)
    nw, nh = int(w * scale), int(h * scale)

    img_resized = cv2.resize(image, (nw, nh))
    canvas = np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    dx, dy = (target_size[0]-nw)//2, (target_size[1]-nh)//2
    canvas[dy:dy+nh, dx:dx+nw] = img_resized
    return canvas

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvid

In [ ]:
def process_and_extract(video_names, sub_path, category_name, label, detector):
    """
    전체 비디오 로드, 체크포인트, 예외 처리가 통합된 풀 버전
    """
    save_dir = os.path.join(GDRIVE_BASE_PATH, category_name)
    os.makedirs(save_dir, exist_ok=True)

    for v_name in tqdm(video_names, desc=f"Processing {category_name}"):
        video_url = SERVER_URL + sub_path + v_name
        video_id = v_name.split('.')[0]
        video_folder = os.path.join(save_dir, video_id)

        # [체크포인트 로직] 이미 15프레임 이상 존재하면 스킵
        if os.path.exists(video_folder) and len(os.listdir(video_folder)) >= NUM_FRAMES:
            continue

        os.makedirs(video_folder, exist_ok=True)

        # 임시 파일을 생성하여 서버에서 영상 다운로드
        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                if total <= 0:
                    print(f"⚠️ {v_name}: 영상을 읽을 수 없습니다.")
                    continue

                # 15프레임을 고르게 추출하기 위한 간격 설정
                interval = max(1, total // NUM_FRAMES)

                last_face_info = None
                video_faces_found = False

                for i in range(NUM_FRAMES):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                    ret, frame = cap.read()
                    if not ret or frame is None:
                        continue

                    # 1. RetinaFace 검출 시도
                    faces = detector.get(frame)
                    face_to_save = None

                    if faces:
                        # 인식 성공 -> 정사각 크롭 (정렬 포함)
                        # score가 없는 경우를 대비해 getattr 사용
                        face_info = sorted(faces, key=lambda x: getattr(x, 'score', 0.0) or 0.0, reverse=True)[0]
                        face_to_save = get_aligned_face(frame, face_info)

                        if face_to_save is not None:
                            last_face_info = face_info
                            video_faces_found = True

                    # 2. 인식 실패 시 -> 이전 프레임 위치 정보 활용
                    if face_to_save is None and last_face_info is not None:
                        face_to_save = get_aligned_face(frame, last_face_info)

                    # 최종 이미지 저장
                    if face_to_save is not None:
                        img_name = f"f{i:03d}_{label}.jpg"
                        cv2.imwrite(os.path.join(video_folder, img_name), face_to_save)

                # 3. [최종 Fallback] 영상 내내 얼굴을 한 번도 못 찾은 경우
                if not video_faces_found:
                    for i in range(NUM_FRAMES):
                        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                        ret, frame = cap.read()
                        if ret and frame is not None:
                            # 얼굴이 없으므로 전체 프레임을 정사각 리사이즈(짜부)하여 저장
                            full_resized = cv2.resize(frame, (256, 256))
                            img_name = f"f{i:03d}_{label}.jpg"
                            cv2.imwrite(os.path.join(video_folder, img_name), full_resized)

                cap.release()

            except Exception as e:
                # 에러 발생 시 폴더가 비어있으면 삭제 (다시 시도할 수 있도록)
                if os.path.exists(video_folder) and not os.listdir(video_folder):
                    os.rmdir(video_folder)
                print(f"❌ Error {v_name}: {e}")

In [ ]:
# --- 실행 부분 ---

# 1. 테스트 모드 설정 (True일 경우 각 카테고리당 2개씩만 샘플링)
IS_TEST_MODE = False

# 데이터셋 리스트 가져오기
real_videos, fake_videos_dict = get_balanced_dataset(is_test=IS_TEST_MODE)
print(f"📊 처리 예정 Real 영상 개수: {len(real_videos)}")

# 2. Real 영상 처리
# detector는 이전에 정의한 RetinaFace 객체(app)를 사용합니다.
real_sub_path = f"original_sequences/youtube/{COMPRESSION}/videos/"
print("\n🎬 [1/2] Real 영상 처리를 시작합니다...")
process_and_extract(real_videos, real_sub_path, "Real", 0, detector)

# 3. Fake 영상 카테고리별 처리
print("\n🎬 [2/2] Fake 영상 카테고리별 처리를 시작합니다...")
for method, v_list in fake_videos_dict.items():
    fake_sub_path = f"manipulated_sequences/{method}/{COMPRESSION}/videos/"
    category_name = f"Fake/{method}"

    print(f"\n🔎 {method} 처리 중... (영상 개수: {len(v_list)})")
    # 카테고리 이름에 따라 폴더 구조가 자동으로 생성됩니다 (예: Fake/Deepfakes)
    process_and_extract(v_list, fake_sub_path, category_name, 1, detector)

print("\n" + "="*50)
print("🎉 모든 전처리가 완료되었습니다!")
print(f"📂 저장 경로: {GDRIVE_BASE_PATH}")
print("="*50)

서버에서 파일 리스트를 가져오는 중...
📊 처리 예정 Real 영상 개수: 1000

🎬 [1/2] Real 영상 처리를 시작합니다...


Processing Real: 100%|██████████| 1000/1000 [2:19:51<00:00,  8.39s/it]



🎬 [2/2] Fake 영상 카테고리별 처리를 시작합니다...

🔎 Deepfakes 처리 중... (영상 개수: 250)


Processing Fake/Deepfakes: 100%|██████████| 250/250 [35:36<00:00,  8.54s/it]



🔎 Face2Face 처리 중... (영상 개수: 250)


...


🔎 FaceSwap 처리 중... (영상 개수: 250)


Processing Fake/FaceSwap: 100%|██████████| 250/250 [33:59<00:00,  8.16s/it]



🔎 NeuralTextures 처리 중... (영상 개수: 250)


Processing Fake/NeuralTextures: 100%|██████████| 250/250 [34:17<00:00,  8.23s/it]


🎉 모든 전처리가 완료되었습니다!
📂 저장 경로: /content/drive/MyDrive/HECTO/Dataset/FF_frames
